# Module 5: Long-Term Memory in LangGraph

**Building on Previous Modules:**
- Module 5.1: Built short-term memory (conversation context)
- Module 5.2: Managed SQLite persistence
- Module 5.3: **Build memory that persists across sessions!**

**What you'll learn:**
- 🧠 Long-term vs Short-term memory
- 💾 LangGraph Store API
- 🗂️ Memory namespaces and organization
- 📊 Semantic memory (facts & profiles)
- 📚 Episodic memory (experiences)
- ⚙️ Procedural memory (rules & prompts)
- 🔍 Memory search and retrieval
- 🎯 Production-ready patterns

**Real HR Use Case:**
Build an HR assistant that:
- Remembers employee preferences across sessions
- Learns from past interactions
- Maintains user profiles
- Adapts its behavior over time

**Time:** 3-4 hours

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and required packages
!pip install --pre -U langchain langchain-openai langgraph langchain-community
!pip install langgraph-checkpoint-sqlite  # For SQLite persistence

## Setup: Configure API Keys & Imports

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Common imports
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Annotated, TypedDict
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain.agents.tool_node import InjectedState, InjectedStore
from langchain.agents import AgentState
from langchain_core.messages import SystemMessage
import json
from datetime import datetime
import uuid

print("✅ Setup complete!")

---
# Part 1: Understanding Long-Term Memory 🧠

## Short-Term vs Long-Term Memory

### Short-Term Memory (Previous Module)
```
Session 1:
User: "My name is Priya"
Agent: "Hello Priya!"

[App restarts]

Session 2 (Same thread):
User: "What's my name?"
Agent: "Your name is Priya!" ✅
```

### Long-Term Memory (This Module)
```
Session 1:
User: "My name is Priya and I prefer morning shifts"
Agent: "Got it! I'll remember your preference."

[Days later, different thread]

Session 2 (Different thread):
User: "Schedule a meeting for me"
Agent: "I'll schedule it in the morning as you prefer!" ✅
```

## Key Differences

| Feature | Short-Term | Long-Term |
|---------|-----------|----------|
| **Scope** | Single thread/session | Across all threads |
| **Storage** | Checkpoints | Store (Database) |
| **Lifespan** | Session duration | Permanent |
| **Use Case** | "What did I just say?" | "What are my preferences?" |
| **Key** | thread_id | user_id/org_id |

## Memory Types

### 1. **Semantic Memory** - Facts & Profiles
- User preferences, settings
- Company policies, employee data
- Example: "Priya prefers email notifications"

### 2. **Episodic Memory** - Experiences
- Past conversations and interactions
- Historical decisions and outcomes
- Example: "Last time we solved this by..."

### 3. **Procedural Memory** - Rules & Behavior
- Custom instructions per user
- Learned workflows
- Example: "Always be formal with executives"

---
# Part 2: LangGraph Store API 💾

**Key Concept:** Store is a key-value database for long-term memories.

**Core Operations:**
- `put()` - Save/Update memory
- `get()` - Retrieve memory
- `search()` - Find memories
- `delete()` - Remove memory

**Organization:**
- **Namespace**: Group related memories (like folders)
- **Key**: Unique identifier within namespace (like filename)

## Lab 1.1: Basic Store Operations

In [ ]:
# Create an in-memory store
store = InMemoryStore()

print("=" * 70)
print("Lab 1.1: Basic Store Operations")
print("=" * 70 + "\n")

# 1. PUT - Save employee profile
employee_profile = {
    "name": "Priya Sharma",
    "employee_id": "101",
    "department": "Engineering",
    "preferences": {
        "notification_method": "email",
        "work_hours": "9 AM - 5 PM",
        "preferred_meeting_time": "morning"
    },
    "leave_balance": 12
}

# Save to store
# Namespace: "employees" (like a folder)
# Key: "user_101" (like a filename)
store.put(
    namespace=("employees",),  # Tuple for hierarchical namespaces
    key="user_101",
    value=employee_profile
)
print("✅ Saved employee profile for Priya")

# 2. GET - Retrieve employee profile
retrieved = store.get(
    namespace=("employees",),
    key="user_101"
)
print(f"\n📥 Retrieved profile:")
print(f"Name: {retrieved.value['name']}")
print(f"Department: {retrieved.value['department']}")
print(f"Preferred meeting time: {retrieved.value['preferences']['preferred_meeting_time']}")

# 3. UPDATE - Modify profile
employee_profile["leave_balance"] = 10  # Used 2 days
store.put(
    namespace=("employees",),
    key="user_101",
    value=employee_profile
)
print("\n✅ Updated leave balance")

# 4. Multiple employees
store.put(
    namespace=("employees",),
    key="user_102",
    value={
        "name": "Rahul Verma",
        "employee_id": "102",
        "department": "Marketing",
        "leave_balance": 8
    }
)
print("✅ Saved second employee profile")

# 5. SEARCH - Find all employees
all_employees = store.search(("employees",))
print(f"\n📊 Total employees in store: {len(all_employees)}")
for emp in all_employees:
    print(f"  - {emp.value['name']} ({emp.value['department']})")

print("\n✅ Store operations complete!")

## Lab 1.2: Hierarchical Namespaces

**Use Case:** Organize memories by company → department → user

In [ ]:
store = InMemoryStore()

print("=" * 70)
print("Lab 1.2: Hierarchical Namespaces")
print("=" * 70 + "\n")

# Hierarchical structure: company → department → user
store.put(
    namespace=("acme_corp", "engineering", "user_101"),
    key="profile",
    value={"name": "Priya Sharma", "role": "Senior Developer"}
)

store.put(
    namespace=("acme_corp", "marketing", "user_102"),
    key="profile",
    value={"name": "Rahul Verma", "role": "Marketing Manager"}
)

store.put(
    namespace=("acme_corp", "engineering", "user_103"),
    key="profile",
    value={"name": "Anjali Patel", "role": "Tech Lead"}
)

print("✅ Saved 3 employee profiles with hierarchical namespaces\n")

# Search by department
print("🔍 Searching Engineering department:")
eng_employees = store.search(("acme_corp", "engineering"))
for emp in eng_employees:
    print(f"  - {emp.value['name']}: {emp.value['role']}")

print("\n🔍 Searching Marketing department:")
marketing_employees = store.search(("acme_corp", "marketing"))
for emp in marketing_employees:
    print(f"  - {emp.value['name']}: {emp.value['role']}")

print("\n🔍 Searching entire company:")
all_employees = store.search(("acme_corp",))
print(f"Total employees: {len(all_employees)}")

print("\n✅ Hierarchical organization makes data management easy!")

---
# Part 3: Semantic Memory - User Profiles 📊

**Use Case:** Remember facts about users across sessions

## Lab 2.1: Reading Long-Term Memory in Tools

In [ ]:
# Initialize store and pre-populate with employee data
store = InMemoryStore()

# Pre-populate store with employee profiles
store.put(
    namespace=("users",),
    key="user_101",
    value={
        "name": "Priya Sharma",
        "employee_id": "101",
        "department": "Engineering",
        "preferences": {
            "notification_method": "email",
            "meeting_time": "morning"
        },
        "leave_balance": 12
    }
)

# Tool that reads from long-term memory
@tool
def get_user_preferences(
    user_id: Annotated[str, "User ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Get user preferences from long-term memory."""
    # Access store in tool!
    user_profile = store.get(namespace=("users",), key=user_id)
    
    if user_profile is None:
        return f"No profile found for {user_id}"
    
    prefs = user_profile.value.get("preferences", {})
    name = user_profile.value.get("name", "Unknown")
    
    return f"""{name}'s preferences:
- Notification method: {prefs.get('notification_method', 'Not set')}
- Preferred meeting time: {prefs.get('meeting_time', 'Not set')}
- Leave balance: {user_profile.value.get('leave_balance', 0)} days
"""

# Create agent with store
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_user_preferences],
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    store=store,  # Pass store to agent!
    system_prompt="You are an HR assistant with access to employee profiles."
)

print("=" * 70)
print("Lab 2.1: Reading Long-Term Memory")
print("=" * 70 + "\n")

# Test 1: Get preferences
result = agent.invoke(
    {"messages": "What are the preferences for user_101?"},
    config={"configurable": {"thread_id": "session_1"}}
)
print("Test 1 - Get preferences:")
print(result['messages'][-1].content)

# Test 2: Different session, same user - memory persists!
result = agent.invoke(
    {"messages": "When should I schedule meetings for user_101?"},
    config={"configurable": {"thread_id": "session_2"}}  # Different thread!
)
print("\nTest 2 - Different session (memory persists):")
print(result['messages'][-1].content)

print("\n✅ Long-term memory works across different sessions!")

## Lab 2.2: Writing Long-Term Memory from Tools

In [ ]:
store = InMemoryStore()

# Schema for user preferences
class UserPreferences(TypedDict):
    notification_method: str  # 'email', 'slack', 'phone'
    meeting_time: str  # 'morning', 'afternoon', 'evening'
    work_style: str  # 'collaborative', 'independent', 'hybrid'

@tool
def save_user_preferences(
    user_id: Annotated[str, "User ID to save preferences for"],
    preferences: Annotated[UserPreferences, "User preferences to save"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Save or update user preferences in long-term memory."""
    # Get existing profile or create new
    existing = store.get(namespace=("users",), key=user_id)
    
    if existing:
        profile = existing.value
        profile["preferences"] = preferences
    else:
        profile = {
            "user_id": user_id,
            "preferences": preferences
        }
    
    # Save to store
    store.put(
        namespace=("users",),
        key=user_id,
        value=profile
    )
    
    return f"✅ Saved preferences for {user_id}: {preferences}"

@tool
def get_user_info(
    user_id: Annotated[str, "User ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Get user information from long-term memory."""
    profile = store.get(namespace=("users",), key=user_id)
    
    if profile is None:
        return f"No profile found for {user_id}"
    
    return f"Profile for {user_id}: {profile.value}"

# Create agent with write capabilities
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[save_user_preferences, get_user_info],
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    store=store,
    system_prompt="""You are an HR assistant that helps users manage their preferences.
    When users tell you their preferences, save them using the save_user_preferences tool."""
)

print("=" * 70)
print("Lab 2.2: Writing Long-Term Memory")
print("=" * 70 + "\n")

config = {"configurable": {"thread_id": "onboarding_session"}}

# User sets preferences
result = agent.invoke({
    "messages": """Hi! I'm user_101. I prefer:
    - Email notifications
    - Morning meetings
    - Independent work style
    Please save these preferences."""
}, config)
print("User sets preferences:")
print(result['messages'][-1].content)

# Different session - verify memory persists
result = agent.invoke(
    {"messages": "What are my preferences? I'm user_101"},
    config={"configurable": {"thread_id": "different_session"}}
)
print("\nDifferent session - retrieve preferences:")
print(result['messages'][-1].content)

print("\n✅ Preferences saved and retrieved across sessions!")

---
# Part 4: Episodic Memory - Learning from Experience 📚

**Use Case:** Remember past interactions to improve responses

## Lab 3.1: Storing Interaction History

In [ ]:
store = InMemoryStore()

@tool
def save_interaction(
    user_id: Annotated[str, "User ID"],
    interaction_type: Annotated[str, "Type: 'question', 'issue', 'feedback'"],
    description: Annotated[str, "Description of interaction"],
    resolution: Annotated[str, "How it was resolved"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Save an interaction to episodic memory."""
    interaction_id = str(uuid.uuid4())[:8]
    
    interaction_data = {
        "interaction_id": interaction_id,
        "type": interaction_type,
        "description": description,
        "resolution": resolution,
        "timestamp": datetime.now().isoformat()
    }
    
    # Save interaction
    store.put(
        namespace=("interactions", user_id),
        key=interaction_id,
        value=interaction_data
    )
    
    return f"✅ Saved interaction {interaction_id}"

@tool
def get_past_interactions(
    user_id: Annotated[str, "User ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Retrieve past interactions for a user."""
    interactions = store.search(namespace=("interactions", user_id))
    
    if not interactions:
        return f"No past interactions found for {user_id}"
    
    result = f"Past interactions for {user_id}:\n\n"
    for item in interactions:
        data = item.value
        result += f"Type: {data['type']}\n"
        result += f"Description: {data['description']}\n"
        result += f"Resolution: {data['resolution']}\n"
        result += f"Date: {data['timestamp'][:10]}\n\n"
    
    return result

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[save_interaction, get_past_interactions],
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    store=store,
    system_prompt="""You are an HR assistant that learns from past interactions.
    Save important interactions so you can reference them later."""
)

print("=" * 70)
print("Lab 3.1: Episodic Memory")
print("=" * 70 + "\n")

config1 = {"configurable": {"thread_id": "issue_session_1"}}

# First interaction - report issue
result = agent.invoke({
    "messages": """I'm user_101. I had an issue accessing the leave portal.
    It kept showing 'Access Denied'. We fixed it by resetting my credentials."""
}, config1)
print("Session 1 - Report issue:")
print(result['messages'][-1].content)

# Different session - similar issue
config2 = {"configurable": {"thread_id": "issue_session_2"}}
result = agent.invoke({
    "messages": """I'm user_101. I'm having trouble accessing the leave portal again.
    What should I do?"""
}, config2)
print("\nSession 2 - Similar issue (learns from past):")
print(result['messages'][-1].content)

print("\n✅ Agent learned from past experience!")

---
# Part 5: Procedural Memory - Adaptive Behavior ⚙️

**Use Case:** Learn and adapt system behavior over time

## Lab 4.1: User-Specific Instructions

In [ ]:
store = InMemoryStore()

# Pre-populate with user-specific instructions
store.put(
    namespace=("instructions",),
    key="user_101",
    value={
        "communication_style": "professional and concise",
        "special_instructions": [
            "Always CC manager on leave requests",
            "Prefer email over chat",
            "Send reminders 2 days before deadlines"
        ]
    }
)

store.put(
    namespace=("instructions",),
    key="user_102",
    value={
        "communication_style": "friendly and detailed",
        "special_instructions": [
            "Provide step-by-step guides",
            "Use examples when explaining"
        ]
    }
)

@tool
def get_user_instructions(
    user_id: Annotated[str, "User ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Get customized instructions for interacting with a specific user."""
    instructions = store.get(namespace=("instructions",), key=user_id)
    
    if instructions is None:
        return "No special instructions for this user."
    
    data = instructions.value
    result = f"Instructions for {user_id}:\n"
    result += f"Communication style: {data['communication_style']}\n"
    result += "Special instructions:\n"
    for instr in data['special_instructions']:
        result += f"  - {instr}\n"
    
    return result

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_user_instructions],
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    store=store,
    system_prompt="""You are an adaptive HR assistant.
    Before responding, check user-specific instructions and adapt your behavior."""
)

print("=" * 70)
print("Lab 4.1: User-Specific Instructions")
print("=" * 70 + "\n")

# Test with user_101 (professional style)
result = agent.invoke({
    "messages": "I'm user_101. How do I apply for leave?"
})
print("User 101 (professional style):")
print(result['messages'][-1].content)

# Test with user_102 (friendly, detailed style)
result = agent.invoke({
    "messages": "I'm user_102. How do I apply for leave?"
})
print("\nUser 102 (friendly, detailed style):")
print(result['messages'][-1].content)

print("\n✅ Agent adapted behavior based on user!")

---
# Part 6: Production Pattern - Complete HR Assistant 🎯

**Combining all memory types for production use**

## Lab 5.1: Complete Production HR Assistant

In [ ]:
# Production HR Assistant with all memory types
store = InMemoryStore()

# Define comprehensive tool suite
@tool
def get_employee_profile(
    user_id: Annotated[str, "Employee ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Get employee profile (Semantic Memory)."""
    profile = store.get(namespace=("profiles",), key=user_id)
    if profile:
        return json.dumps(profile.value, indent=2)
    return f"No profile found for {user_id}"

@tool
def save_employee_info(
    user_id: Annotated[str, "Employee ID"],
    info: Annotated[dict, "Employee information"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Save employee information."""
    store.put(namespace=("profiles",), key=user_id, value=info)
    return f"✅ Saved info for {user_id}"

@tool
def log_interaction(
    user_id: Annotated[str, "Employee ID"],
    interaction_summary: Annotated[str, "Brief summary"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Log interaction (Episodic Memory)."""
    interaction_id = str(uuid.uuid4())[:8]
    store.put(
        namespace=("interactions", user_id),
        key=interaction_id,
        value={
            "summary": interaction_summary,
            "timestamp": datetime.now().isoformat()
        }
    )
    return f"✅ Logged interaction {interaction_id}"

@tool
def get_user_instructions(
    user_id: Annotated[str, "Employee ID"],
    store: Annotated[InMemoryStore, InjectedStore]
) -> str:
    """Get user-specific instructions (Procedural Memory)."""
    instructions = store.get(namespace=("instructions",), key=user_id)
    if instructions:
        return json.dumps(instructions.value, indent=2)
    return "No special instructions"

# Create production agent
production_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[
        get_employee_profile,
        save_employee_info,
        log_interaction,
        get_user_instructions
    ],
    checkpointer=SqliteSaver.from_conn_string(":memory:"),
    store=store,
    system_prompt="""You are a production HR assistant with:
    
    - Semantic Memory: Employee profiles and preferences
    - Episodic Memory: Past interactions and resolutions
    - Procedural Memory: User-specific behavioral instructions
    
    Before responding:
    1. Check user profile for context
    2. Check user instructions for preferences
    3. Log important interactions
    
    Be helpful, professional, and personalized."""
)

print("=" * 70)
print("Lab 5.1: Production HR Assistant")
print("=" * 70 + "\n")

# Simulate multi-session usage
# Session 1: Onboarding
result = production_agent.invoke(
    {"messages": "Hi! I'm new employee user_201. I'm Priya from Engineering."},
    config={"configurable": {"thread_id": "onboarding_201"}}
)
print("Session 1 (Onboarding):")
print(result['messages'][-1].content)

# Session 2: Different day, different thread
result = production_agent.invoke(
    {"messages": "This is user_201. How do I apply for leave?"},
    config={"configurable": {"thread_id": "leave_query_201"}}
)
print("\nSession 2 (Different day - remembers user):")
print(result['messages'][-1].content)

print("\n✅ Production agent with complete long-term memory!")

---
# Summary & Best Practices

## Long-Term Memory Comparison

| Memory Type | What to Store | When to Use | Example |
|-------------|--------------|-------------|----------|
| **Semantic** | Facts, profiles | User preferences | "Prefers email" |
| **Episodic** | Past interactions | Learning from experience | "Fixed by resetting" |
| **Procedural** | Behavior rules | Personalization | "Be formal with execs" |

## Storage Patterns

### Profile Pattern
- ✅ **Pros:** All info in one place, easy to understand
- ❌ **Cons:** Can become large, harder to update
- **Use when:** User data is well-scoped and stable

### Collection Pattern
- ✅ **Pros:** Flexible, easier to add new items
- ❌ **Cons:** Need to manage search and updates
- **Use when:** Data grows over time (interactions, examples)

## Production Checklist

✅ **Use persistent store** (not InMemoryStore in production)  
✅ **Organize with namespaces** (user_id, org_id, dept)  
✅ **Combine with short-term memory** (checkpointer + store)  
✅ **Implement search** for large datasets  
✅ **Add cleanup policies** (delete old data)  
✅ **Monitor store size** and performance  
✅ **Handle missing data** gracefully  

## When to Write Memory

### Hot Path (Real-time)
- ✅ **Pros:** Immediate updates, transparent to user
- ❌ **Cons:** Adds latency, agent multitasking
- **Use when:** User needs confirmation ("Saved your preference!")

### Background (Async)
- ✅ **Pros:** No latency, focused task completion
- ❌ **Cons:** Delayed updates, complexity
- **Use when:** Processing large conversations, batch updates

## Key Takeaways

1. **LangGraph Store**
   - Key-value database for long-term memory
   - Hierarchical namespaces for organization
   - Cross-session persistence

2. **Memory Types**
   - Semantic: Facts and profiles
   - Episodic: Past experiences
   - Procedural: Learned behavior

3. **Production Patterns**
   - Combine short-term (checkpoints) + long-term (store)
   - Use proper namespaces for organization
   - Implement search for large datasets
   - Choose storage pattern based on use case

## Next: Transition to LangChain

In the next module, we'll explore how to implement similar patterns using pure LangChain with custom backends.

---

**Remember:** Long-term memory makes agents truly useful!

# Exercises

## Exercise 1: Company Knowledge Base
Build a system that stores:
- Company policies (semantic)
- Past Q&A sessions (episodic)
- Department-specific guidelines (procedural)

## Exercise 2: Learning from Feedback
Implement a system that:
- Collects user feedback
- Updates agent behavior
- Tracks improvement over time

## Exercise 3: Multi-Tenant System
Create a system supporting multiple organizations:
- Separate namespaces per organization
- Shared knowledge base
- Org-specific customization

## Exercise 4: Memory Analytics
Build analytics for your memory system:
- Most accessed profiles
- Common interaction patterns
- Memory growth over time

## Bonus: Semantic Search
Implement semantic search using embeddings:
- Store embeddings with memories
- Find similar past interactions
- Retrieve relevant examples